In [10]:
loan_data = {
     "C123": {
       "customer_name": "Ravi Kumar",
       "total_due": 8400,
       "due_date": "2025-01-15",
       "dpd": 32
     },
     "C456": {
       "customer_name": "Aditi Sharma",
       "total_due": 15200,
       "due_date": "2025-02-01",
       "dpd": 5
     }
   }

In [ ]:
# EXAMPLE CONVERSATION
# Bot: Hello, this is Jiya calling from ABC Finance. Am I speaking with Ravi Kumar?

# User: Yes, speaking.

# Bot: Thank you for confirming, Ravi. Your loan amount of rupees 20000 is pending and overdue for 50 days. When can you make the payment?

# User: I can pay this Friday.

# Bot: Thank you, Ravi. I will note that. Please ensure payment is done on Friday

In [9]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("groq:llama-3.1-8b-instant")


In [25]:
from langchain.tools import tool

@tool
def get_customer_details(customer_id: str) -> str:
    """
    Fetches user details for a customer ID and name from JSON data.
    The chatbot should ask like "Hello, this is Jiya calling from ABC Finance. Am I speaking with {customer_name}?"
    """
    if customer_id in loan_data:
        return loan_data[customer_id]
    else:
        return "Customer ID not found."


@tool
def get_loan_details(customer_id: str, customer_name: str) -> str:
    """
    Fetches user confirmation for a customer ID and name from JSON data. 
    If the user confirms their identity by yes or related word, then  fetch the due amount. For example: total_due": 8400,
    The chatbot should ask like "Thank you for confirming, {customer_name}. Your loan amount of rupees {total_due} is pending from {due_date}. When can you make the payment?
    """
    if customer_id in loan_data and loan_data[customer_id]["customer_name"] == customer_name:
        return loan_data[customer_id]["total_due"]
    else:
        return "No due amount found."

@tool
def record_commitment(customer_id: str, commitment_date: str):
    """Records the user's payment commitment date."""
    return f"Commitment for {customer_id} noted for {commitment_date}"



In [26]:
from langchain.agents import create_agent

prompt = """
You are Jiya from ABC Finance. You MUST follow these conversation rules:

1. Step 1: Ask if you are speaking with the correct customer (use get_customer_name).
2. Step 2: Wait for user to say “yes” before using get_loan_details.
3. Step 3: After confirmation, fetch loan details using get_loan_details.
4. Step 4: Tell them the due amount and ask when they can make the payment.
5. Step 5: When user provides a date, call record_commitment.
6. Chat must be polite and compliant.

Follow steps strictly.
"""
tools = [get_customer_details, get_loan_details, record_commitment]
agent = create_agent(llm, tools=tools, system_prompt=prompt)



In [20]:
query = "What is the total due for customer C123?"
for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the total due for customer C123?
================================== Ai Message ==================================
Tool Calls:
  get_customer_details (p4evgcmt7)
 Call ID: p4evgcmt7
  Args:
    customer_id: C123
  get_user_confirmation (hvnrtetxr)
 Call ID: hvnrtetxr
  Args:
    customer_id: C123
    customer_name: John
================================= Tool Message =================================
Name: get_user_confirmation

No due amount found.
================================== Ai Message ==================================
Tool Calls:
  get_customer_details (17rfta3ah)
 Call ID: 17rfta3ah
  Args:
    customer_id: C123
  get_user_confirmation (53sk0eqd1)
 Call ID: 53sk0eqd1
  Args:
    customer_id: C123
    customer_name: John
================================= Tool Message =================================
Name: get_user_confirmation

No due amount found.
================================== Ai M